In [4]:
# MASTER ONLY
# imports %%solution, %%submission, %%template
%load_ext prog_edu_assistant_tools.magics
from prog_edu_assistant_tools.magics import report, autotest

The prog_edu_assistant_tools.magics extension is already loaded. To reload it, use:
  %reload_ext prog_edu_assistant_tools.magics


# Introduction to NLP (Natural Language Processing)

```
# ASSIGNMENT METADATA
assignment_id: "IntroToNLP"
```

In [42]:
# This is some dummy context
pass

TODO(jnavarroh): Add explanations

## Exercise: Count bigram frequency

```
# EXERCISE METADATA
exercise_id: "BigramFrequency"
```

Using the text database that you have downloaded above, compute the bigram frequencies, and define a function that returns the top k bigrams as a map of bigram (two unigrams joined by a single space) to its occurrence count.

In [45]:
%%solution
def TopBigramFrequency(k):
    """ # BEGIN PROMPT
    # ... put your program here
    """ # END PROMPT
    # BEGIN SOLUTION
    return {'in the': 100, 'something else': 1}
    # END SOLUTION

In [46]:
%%studenttest StudentTest
assert TopBigramFrequency(2)['in the'] == 100

In [47]:
%%inlinetest AutograderTest
assert 'TopBigramFrequency' in globals(), "Did you define a function named 'TopBigramFrequency' in the solution cell?"
assert str(TopBigramFrequency.__class__) == "<class 'function'>", "Did you define a function named 'TopBigramFrequency'? There was a %s instead" % TopBigramFrequency.__class__
ans = TopBigramFrequency(2)
assert 'in the' in ans, "Your computed answer does not have bigram 'in the', but it should be there"
assert ans['in the'] == 100, "Frequency of 'in the' expected to be 100, but got %d" % (ans['in the'])

In [48]:
result, log = %autotest AutograderTest
report(AutograderTest, results=result.results)

In [49]:
%%submission
TopBigramFrequency = 1

In [50]:
result, log = %autotest AutograderTest
report(AutograderTest, results=result.results)